# Training an Object Detector for Duckietown
By orlando.m.bol@gmail.com

YOLO is a fast object detector with decent accuracy. It is real-time and thus useful for the Duckietown environment. For more information, see https://pjreddie.com/darknet/yolo/.

## Clone a forked version of the YOLO3 repo that contains the Duckietown datasets

In [0]:
!git clone https://github.com/TECYL/darknet.git

fatal: destination path 'darknet' already exists and is not an empty directory.


## Install CUDA and compile Darknet (which YOLO is built on)

In [0]:
!wget -O cuda_9.2.88_396.26_linux.run -c https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda_9.2.88_396.26_linux

--2020-05-06 09:47:18--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda_9.2.88_396.26_linux
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda_9.2.88_396.26_linux.run?flGlb4_oNmRA6Q7Cwrp50nvb5XxmIAwwcPJtxLpHVTcCAmkTG_qQzvv4eT_J2IjuBNg2N7HLxUCAlqTDojpsdXrpEj4yni0QwyQuG3Zvv1Q326vxi_We110X1S4QMZWI5CfV2X1lAJ-c45C0yTjR36mJdv6caagOa47rXzhlbGsVieHdbQ2JOOM [following]
--2020-05-06 09:47:19--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda_9.2.88_396.26_linux.run?flGlb4_oNmRA6Q7Cwrp50nvb5XxmIAwwcPJtxLpHVTcCAmkTG_qQzvv4eT_J2IjuBNg2N7HLxUCAlqTDojpsdXrpEj4yni0QwyQuG3Zvv1Q326vxi_We110X1S4QMZWI5CfV2X1lAJ-c45C0yTjR36mJdv6caagOa47rXzhlbGsVieHdbQ2JOOM
Resolving developer.download.nv

In [0]:
!chmod +x cuda_9.2.88_396.26_linux.run
!./cuda_9.2.88_396.26_linux.run --verbose --silent --toolkit --override

Installing the CUDA Toolkit in /usr/local/cuda-9.2 ...
Verifying archive integrity... All good.

Uncompressing NVIDIA CUDA..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

**Include cuda in the PATH and compile Darknet**

Note that the Makefile has been modified to compile with CUDA and without OPENCV

In [0]:
import os
os.environ['PATH'] += ':/usr/local/cuda-9.2/bin'

In [0]:
cd darknet

/content/darknet


In [0]:
!make

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/gemm.c -o obj/gemm.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/utils.c -o obj/utils.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/cuda.c -o obj/cuda.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/deconvolutional_layer.c -o obj/deconvolutional_layer.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/convolutional_layer.c -o obj/convolutional_layer.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -W

## Test YOLO installation

In [0]:
# Download pre-trained weights
!wget https://pjreddie.com/media/files/yolov3.weights

--2018-12-21 23:46:48--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.3.39
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  22.3MB/s    in 11s     

2018-12-21 23:47:00 (21.3 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [0]:
# Run object detection on sample image
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.2/lib64 && ./darknet detect cfg/yolov3-tiny-p4.cfg.cfg yolov3.weights data/dog.jpg

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    8                 152 x 

## Train tiny YOLO on Duckietown images
We realized that the more shallow version of YOLO, named tiny YOLO, is good enough for Duckietown. Since it has fewer convolutional layers, it trains and predicts faster. You can stop the training when you like and use the saved weights to do inference.

In [0]:
# Download pre-trained weights to effectively do transfer learning
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-05-06 09:51:46--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  74.3KB/s    in 38m 51s 

2020-05-06 10:30:39 (68.1 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [0]:
# Create a directory where we will save backups of the weights
!mkdir duckie_backup

In [0]:
# Train tiny YOLO using the already created configuration files
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.2/lib64 && ./darknet detector train cfg/duckie-multi-p4.data cfg/yolov3-tiny-p4.cfg darknet53.conv.74

Streaming output truncated to the last 5000 lines.
52805: 0.060252, 0.057694 avg, 0.001000 rate, 0.029594 seconds, 52805 images
Loaded: 0.000683 seconds
52806: 0.045196, 0.056444 avg, 0.001000 rate, 0.035372 seconds, 52806 images
Loaded: 0.001183 seconds
52807: 0.047230, 0.055523 avg, 0.001000 rate, 0.035186 seconds, 52807 images
Loaded: 0.002025 seconds
52808: 0.064860, 0.056456 avg, 0.001000 rate, 0.033698 seconds, 52808 images
Loaded: 0.000038 seconds
52809: 0.049585, 0.055769 avg, 0.001000 rate, 0.036205 seconds, 52809 images
Loaded: 0.000039 seconds
52810: 0.069634, 0.057156 avg, 0.001000 rate, 0.037216 seconds, 52810 images
Resizing
544
Loaded: 0.000052 seconds
52811: 0.045639, 0.056004 avg, 0.001000 rate, 0.016470 seconds, 52811 images
Loaded: 0.011102 seconds
52812: 0.021411, 0.052545 avg, 0.001000 rate, 0.019411 seconds, 52812 images
Loaded: 0.005436 seconds
52813: 0.110751, 0.058365 avg, 0.001000 rate, 0.028388 seconds, 52813 images
Loaded: 0.000028 seconds
52814: 0.037339, 0

## Test your Duckietown Object Detector
Pick any image from the directory `duckiestuff/testset`

In [0]:
# Let's see what has been saved during training
!ls -hlt duckie_backup/

total 795M
-rw-r--r-- 1 root root 34M May  5 14:06 yolov3-tiny-p4.backup
-rw-r--r-- 1 root root 34M May  5 14:05 yolov3-tiny-p4_140000.weights
-rw-r--r-- 1 root root 34M May  5 14:01 yolov3-tiny-p4_130000.weights
-rw-r--r-- 1 root root 34M May  5 13:56 yolov3-tiny-p4_120000.weights
-rw-r--r-- 1 root root 34M May  5 13:52 yolov3-tiny-p4_110000.weights
-rw-r--r-- 1 root root 34M May  5 13:48 yolov3-tiny-p4_100000.weights
-rw-r--r-- 1 root root 34M May  5 13:43 yolov3-tiny-p4_90000.weights
-rw-r--r-- 1 root root 34M May  5 13:39 yolov3-tiny-p4_80000.weights
-rw-r--r-- 1 root root 34M May  5 13:35 yolov3-tiny-p4_70000.weights
-rw-r--r-- 1 root root 34M May  5 13:30 yolov3-tiny-p4_60000.weights
-rw-r--r-- 1 root root 34M May  5 13:26 yolov3-tiny-p4_50000.weights
-rw-r--r-- 1 root root 34M May  5 13:22 yolov3-tiny-p4_40000.weights
-rw-r--r-- 1 root root 34M May  5 13:18 yolov3-tiny-p4_30000.weights
-rw-r--r-- 1 root root 34M May  5 13:14 yolov3-tiny-p4_20000.weights
-rw-r--r-- 1 root root 34

In [1]:
# Run inference
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.2/lib64 && ./darknet detector test cfg/duckie-multi-p4.data cfg/yolov3-tiny-p4.cfg duckie_backup/yolov3-tiny-p4.backup data_folder/testset/frame0115.jpg -thresh .50

/bin/bash: ./darknet: No such file or directory


## Transferring weights to Google Drive
You will want to transfer the trained weights to some other computer so that you can run inference. The easiest way is to copy them to your Google drive and then download the weights from there.

In [0]:
# Map your Google drive to Collab
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Copy weights to google drive
!cp duckie_backup/yolov3-tiny-p4.backup /content/gdrive/My\ Drive/yolov3-tiny-p4.weights